In [ ]:
import sys
import pandas
sys.path.append("..") 
sys.path.append("../src") 
from src import dataFiles, dataMangling, dataPlotting, dataTable

In [ ]:
ts, bnn = dataFiles.data(withSynthetic=True)
dates = dataMangling.dates_list(ts)
ts_BuLa, Bundeslaender = dataMangling.join_tables_for_and_aggregate_Bundeslaender(ts, bnn)

In [ ]:
"\n".join(ts[ts.ADMIN.isna()]["AGS"].tolist())

In [ ]:
Bundeslaender

In [ ]:
daily, cumulative, title, filename, population = dataMangling.get_BuLa(Bundeslaender, "Hessen")

In [ ]:
center = dataMangling.temporal_center(daily)[0]
center

In [ ]:
dataMangling.temporal_center(dataMangling.get_BuLa(Bundeslaender, "Hessen")[0])[0]

In [ ]:
Bundeslaender.index.tolist()

In [ ]:
Bundeslaender["centerday"] = [dataMangling.temporal_center(dataMangling.get_BuLa(Bundeslaender, name)[0])[0]
                                  for name in Bundeslaender.index.tolist() ]

In [ ]:
Bundeslaender

In [ ]:
Bundeslaender.sort_values("centerday",inplace=True)

In [ ]:
Bundeslaender

In [ ]:
Bundeslaender.loc['Deutschland'] = Bundeslaender.sum().values.tolist()

In [ ]:
ts

In [ ]:
ts.drop(ts[ts.ADMIN.isna()].index, inplace=True)

ts

In [ ]:
import io, sys

import pandas
import requests

sys.path.append("..") 
sys.path.append("../src") 

from dataFiles import OPENDATASOFT_URL01, OPENDATASOFT_URL02, DISTANCES_PATH 

def download_kreise_locations(url1=OPENDATASOFT_URL01, url2=OPENDATASOFT_URL02):
    print ("Downloading large table with Kreise locations. For infos see")
    print (url1)
    print ("Patience please ...", end=" ")
    s=requests.get(url2).content
    LKG=pandas.read_csv(io.StringIO(s.decode('utf-8')), sep=';') # error_bad_lines=False)
    # immediately drop non-Kreis row(s)
    LKG.dropna(subset=['Cca 2'],inplace=True)
    # turn AGS into integer:
    LKG["Cca 2"]=LKG["Cca 2"].astype(int)
    print ("Done downloading.")
    return LKG
LKG=download_kreise_locations()

In [ ]:
LKG

In [ ]:
# accelerate lookup
AGS_to_geopoint=dict(LKG[["Cca 2","Geo Point"]].set_index("Cca 2").to_dict('series')["Geo Point"])
AGS_to_geopoint

In [ ]:
import geopy.distance

def geo_distance(AGS_to_geopoint, AGS1, AGS2):
    c1 = list(map(float,AGS_to_geopoint[AGS1].split(",")))
    c2 = list(map(float,AGS_to_geopoint[AGS2].split(",")))
    # print (c1, c2 )
    return geopy.distance.geodesic(c1, c2).kilometers

print (geo_distance(AGS_to_geopoint, 1001, 1002))

In [ ]:
def make_distances_table(AGS_to_geopoint, all_AGS, filename=DISTANCES_PATH):
    headers = ['AGS1','AGS2','km']
    counter,countermax=0, len(all_AGS)*(len(all_AGS)-1)
    distances=pandas.DataFrame([], columns = headers)
    for AGS1 in all_AGS:
        for AGS2 in all_AGS:
            if AGS1>=AGS2:
                continue
            geodist = geo_distance(AGS_to_geopoint, AGS1, AGS2)
            row=pandas.Series([AGS1, AGS2, geodist], index=headers)
            distances=distances.append(row, ignore_index=True)
            row=pandas.Series([AGS2, AGS1, geodist], index=headers)
            distances=distances.append(row, ignore_index=True)
            counter+=2
            if not counter%100:
                print (counter, "/", countermax)

    print ("Done calculating distances. Len(table)=%d" % len(distances))

    distances["AGS1"]=distances["AGS1"].astype(int)
    distances["AGS2"]=distances["AGS2"].astype(int)
        
    print ("Distances are distributed like this:")
    print(distances["km"].describe())
    
    distances.sort_values("km", inplace=True)
    print("Done sorting.")
    
    distances.to_csv(filename)
    print ("Saved to", filename)
    return distances, filename

all_AGS=LKG["Cca 2"].dropna().astype(int).tolist()
# all_AGS=all_AGS[:20] # reduce number for dev'ing
print(all_AGS)
print()

distances, filename = make_distances_table(AGS_to_geopoint, all_AGS)
distances

In [ ]:
distances

In [ ]:
(distances[(distances.km<150) & (distances.AGS1==9371)])

In [ ]:
import sys
sys.path.append("..") 
sys.path.append("../src") 
import dataFiles, dataMangling, dataPlotting, districtDistances
distances = districtDistances.load_distances()
print (districtDistances.nearby(distances, 1001, 50))


In [ ]:
distances[(distances.km<=50)]

In [ ]:
ts, bnn = dataFiles.data(withSynthetic=True)

In [ ]:
bnn.BEZ.unique()

In [ ]:
LKG

In [ ]:
def compare_risklayer_with_opendatasoft(bnn):
    filename = dataFiles.OPENDATASOFT_PATH
    LKG = districtDistances.load_kreise_locations(filename)
    ODS = set(LKG["Cca 2"].tolist())
    ts, bnn = dataFiles.data(withSynthetic=True)
    RSL=set(bnn["AGS"].values.tolist())
    
    diff1 = list(ODS-RSL)
    names1=([LKG["Name 2"][LKG["Cca 2"]==AGS].tolist()[0] for AGS in diff1])
    print ("ODS-RSL: %s = %s"% (diff1, names1))
    diff2 = list(RSL-ODS)
    names2=([bnn["GEN"][bnn["AGS"]==AGS].tolist()[0] for AGS in diff2])
    print ("RSL-ODS: %s = %s"% (diff2, names2))
compare_risklayer_with_opendatasoft(bnn)

In [ ]:
i=LKG[LKG["Cca 2"]==3152].index.tolist()
if i:
    LKG.at[i, "Cca 2"] = 3159

In [ ]:
LKG.loc[i]

In [ ]:
i=LKG[LKG["Cca 2"]==3156].index.tolist()
if i:
    LKG.drop(index=i, inplace=True)

In [ ]:
LKG[(LKG["Cca 2"]> 3150) & (LKG["Cca 2"]< 3161)]

In [ ]:
a=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0]
list(map(int, a))

In [ ]:
import sys
sys.path.append("..") 
sys.path.append("../src") 
import dataFiles, dataMangling, dataPlotting, districtDistances,dataTable
ts, bnn, ts_sorted, Bundeslaender_sorted, dates, datacolumns = dataMangling.dataMangled()
distances = districtDistances.load_distances()
cmap = dataTable.colormap()


In [ ]:
Bundeslaender_sorted.index.tolist()


In [ ]:
DE=Bundeslaender_sorted.drop(["Deutschland", "Dummyland"]).sum()
DE[datacolumns].astype(int).tolist()

In [ ]:
DE = Bundeslaender_sorted.sum()

In [ ]:
DE["Population"]

In [24]:
import sys
import pandas
sys.path.append("..") 
sys.path.append("../src") 
import dataFiles, dataMangling, dataPlotting, districtDistances,dataTable
dataFiles.TS_NEWEST

'../data/GermanyValues_RiskLayer.csv'

In [27]:
ts=pandas.read_csv(dataFiles.TS_NEWEST, encoding='cp1252') # encoding='utf-8')

In [28]:
ts

,AGS,ADMIN,05.03.2020,06.03.2020,07.03.2020,08.03.2020,09.03.2020,10.03.2020,11.03.2020,12.03.20203,...,20.04.2020,21.04.2020,22.04.2020,23.04.2020,24.04.2020,25.04.2020,26.04.2020,27.04.2020,28.04.2020,29.04.2020
0,07131,Ahrweiler (Landkreis),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,115.0,115.0,116.0,116.0,118.0,118.0,118.0,118.0,118.0,121.0
1,09771,Aichach-Friedberg (Landkreis),1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,262.0,272.0,273.0,278.0,278.0,283.0,283.0,287.0,288.0,293.0
2,08425,Alb-Donau-Kreis (Landkreis),2.0,2.0,2.0,3.0,3.0,3.0,3.0,6.0,...,495.0,489.0,501.0,514.0,526.0,533.0,536.0,537.0,537.0,545.0
3,16077,Altenburger Land (Landkreis),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.0,45.0,45.0,45.0,47.0,47.0,47.0,47.0,47.0,48.0
4,07132,Altenkirchen (Westerwald) (Landkreis),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,128.0,128.0,128.0,134.0,137.0,138.0,138.0,143.0,144.0,145.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,07320,Zweibrücken (Kreisfreie Stadt),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0
400,14524,Zwickau (Landkreis),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,697.0,707.0,709.0,717.0,724.0,730.0,738.0,738.0,741.0,745.0
401,Data can be used for reproduction with attribu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,Data sources can be found under https://docs.g...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
